# Mini Fashion MNIST Mini Project
Build your own CNN on Fashion MNIST following the hints.

In [ ]:
# --- 1. Imports & Dataset ---
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# --- 2. Load and preprocess dataset ---
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train[..., None] # (...) means “all existing dimensions” = (num_samples, 28, 28)
X_test = X_test[..., None]

print('Training samples:', X_train.shape[0])
print('Test samples:', X_test.shape[0])

In [ ]:
# --- 3. Build a Simple CNN ---

model = models.Sequential([
    # First convolution + max pooling
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),

    # Second convolution + max pooling
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    # Flatten and fully connected layers
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # output layer for 10 classes
])


In [ ]:
# --- 4. Compile the Model ---
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# --- 5. Train the Model ---
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
# --- 6. Evaluate the Model ---
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc:.4f}')

In [ ]:
# --- 7. Plot Training History ---
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training History')
plt.show()

In [ ]:
# --- 8. Make a Prediction  ---
idx = np.random.randint(len(X_test))
plt.imshow(X_test[idx].reshape(28,28), cmap='gray')
plt.title(f'True label: {y_test[idx]}')
plt.show()
pred = model.predict(X_test[idx:idx+1])
print('Predicted class:', np.argmax(pred))

In [ ]:
# --- 9. Optional Exploration ---
from tensorflow.keras.layers import LeakyReLU

# 1️⃣ Using Dropout
model1 = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # 50% dropout
    layers.Dense(10, activation='softmax')
])

# 2️⃣ Using LeakyReLU activation
model2 = models.Sequential([
    layers.Conv2D(32, (3,3)),
    LeakyReLU(alpha=0.1),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3)),
    LeakyReLU(alpha=0.1),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64),
    LeakyReLU(alpha=0.1),
    layers.Dense(10, activation='softmax')
])



In [ ]:
# you can change between model1 & model2
model2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model2.fit(
    X_train, y_train,
    epochs=10,      # longer training
    batch_size=128, # larger batch size
    validation_data=(X_test, y_test)
)


In [ ]:
# Evaluate the trained model on test set
test_loss, test_accuracy = model2.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


In [ ]:
# Predict classes for test images
predictions = model1.predict(X_test)

# Convert softmax probabilities to class indices
predicted_classes = np.argmax(predictions, axis=1)

# Compare first 10 predictions to true labels
for i in range(10):
    print(f"True: {y_test[i]}, Predicted: {predicted_classes[i]}")


In [ ]:
#Visualize Some Predictions
class_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Show first 5 test images with predictions
for i in range(5):
    plt.imshow(X_test[i].reshape(28,28), cmap='gray')
    plt.title(f"True: {class_labels[y_test[i]]}, Pred: {class_labels[predicted_classes[i]]}")
    plt.show()


In [ ]:
# --- 10. Test Your Model on a New Image ---

from PIL import Image

# Step 1: Load an image
# Make sure it is a fashion item (t-shirt, shoe, etc.)
# Convert to grayscale ('L') since Fashion MNIST is grayscale
img = Image.open("your_image.jpg").convert('L')  

# Step 2: Resize to 28x28 (same as training images)
img = img.resize((28, 28))

# Step 3: Convert to numpy array and normalize
img_array = np.array(img) / 255.0  # scale to 0-1
img_array = img_array.reshape(1, 28, 28, 1)  # shape: (1,28,28,1)

# Step 4: Make a prediction
pred = model.predict(img_array)
predicted_class = np.argmax(pred)
print("Predicted class:", predicted_class)

# Step 5: Visualize the image
import matplotlib.pyplot as plt
plt.imshow(img_array.reshape(28,28), cmap='gray')
plt.title(f"Predicted class: {predicted_class}")
plt.show()
